In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [54]:
train_X = pd.read_csv('train_X.csv')
train_y = pd.read_csv('train_y.csv')
train_X.head()

,contract_id,report_date,specialization_id,contract_init_sum,contract_date,project_id,building_id,contractor_id,contract_current_sum,agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME,...,agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_48M,agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__ALL_TIME,agg_tender_proposal__g_contractor__id__ALL__countDistinct__1W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__2W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__4W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__12W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME
0,5433,2023-01-01,18,1.115267,2022-05-10 21:00:00,43,701,438,0.793952,-0.539030,...,-0.158471,-0.158473,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
1,6875,2023-01-01,18,1.608002,2022-08-21 21:00:00,31,268,438,2.681675,0.743673,...,-0.158471,-0.158473,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
2,1476,2023-01-01,18,-0.360764,2022-10-17 21:00:00,31,268,438,-0.416432,-0.539030,...,-0.158471,-0.158473,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
3,4469,2023-01-01,12,-0.089303,2022-04-20 21:00:00,43,697,484,-0.193827,-0.539030,...,-0.158470,-0.158471,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
4,1330,2023-01-01,12,-0.515778,2022-03-02 21:00:00,49,224,484,0.133446,0.743673,...,-0.158470,-0.158471,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454


In [55]:
train_y.head()

,contract_id,report_date,default6
0,5433,2023-01-01,0
1,6875,2023-01-01,0
2,1476,2023-01-01,0
3,4469,2023-01-01,0
4,1330,2023-01-01,0


In [56]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28831 entries, 0 to 28830
Columns: 169 entries, contract_id to agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME
dtypes: float64(162), int64(5), object(2)
memory usage: 37.2+ MB


In [57]:
quantile_lower = train_X.contract_current_sum.quantile(0.05)
quantile_higher = train_X.contract_current_sum.quantile(0.95)

train_X = train_X.dropna(subset=['agg_BoardOfDirectors__g_contractor__Name__count__ALL_TIME', 'contract_id', 'report_date', 'agg_Finance__g_contractor__Value__LongLiabilities__last__ALL_TIME', 'agg_Finance__g_contractor__Value__ShortLiabilities__last__ALL_TIME', 'contract_current_sum', 'agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME', 'agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME', 'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__ALL_TIME', 'agg_Finance__g_contractor__Value__FixedAssets__last__ALL_TIME', 'agg_scontrol__g_contractor__close_delay__defect_type_tech_supervision__mean__ALL_TIME'])
train_X = train_X[(train_X['agg_Finance__g_contractor__Value__NetProfit_y__last__ALL_TIME'] < train_X['agg_Finance__g_contractor__Value__NetProfit_y__last__ALL_TIME'].quantile(0.95)) & (train_X['agg_Finance__g_contractor__Value__NetProfit_y__last__ALL_TIME'] > train_X['agg_Finance__g_contractor__Value__NetProfit_y__last__ALL_TIME'].quantile(0.05))]
train_X = train_X[(train_X.contract_current_sum > quantile_lower) & (train_X.contract_current_sum < quantile_higher)]
train_X

,contract_id,report_date,specialization_id,contract_init_sum,contract_date,project_id,building_id,contractor_id,contract_current_sum,agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME,...,agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_48M,agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__ALL_TIME,agg_tender_proposal__g_contractor__id__ALL__countDistinct__1W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__2W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__4W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__12W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W,agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME
0,5433,2023-01-01,18,1.115267,2022-05-10 21:00:00,43,701,438,0.793952,-0.539030,...,-0.158471,-0.158473,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
2,1476,2023-01-01,18,-0.360764,2022-10-17 21:00:00,31,268,438,-0.416432,-0.539030,...,-0.158471,-0.158473,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
3,4469,2023-01-01,12,-0.089303,2022-04-20 21:00:00,43,697,484,-0.193827,-0.539030,...,-0.158470,-0.158471,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
4,1330,2023-01-01,12,-0.515778,2022-03-02 21:00:00,49,224,484,0.133446,0.743673,...,-0.158470,-0.158471,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
11,4218,2023-01-01,6,1.410171,2022-11-10 21:00:00,44,728,484,1.035780,-0.539030,...,-0.158470,-0.158471,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28814,5271,2023-07-23,10,-0.373126,2022-07-03 21:00:00,42,915,789,-0.426569,-0.539030,...,-0.158471,-0.158474,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
28815,3382,2023-07-23,21,-0.073025,2023-06-15 21:00:00,47,620,855,-0.180479,-0.539030,...,-0.158471,-0.158472,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
28819,5465,2023-07-23,21,0.090307,2023-05-04 21:00:00,47,621,855,-0.046542,-0.539030,...,-0.158471,-0.158472,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454
28820,6647,2023-07-23,17,1.009716,2022-11-15 21:00:00,45,533,855,0.610944,0.743673,...,-0.158471,-0.158472,-0.136894,-0.157822,-0.178045,-0.190966,-0.200348,-0.225724,-0.256704,-0.258454


In [58]:
train_X.isna().sum()

contract_id                                                            0
report_date                                                            0
specialization_id                                                      0
contract_init_sum                                                      0
contract_date                                                          0
                                                                      ..
agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W          0
agg_tender_proposal__g_contractor__id__ALL__countDistinct__12W         0
agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W         0
agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W         0
agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME    0
Length: 169, dtype: int64

In [59]:
f = train_X.isna().sum().to_frame("Nans").reset_index()
s = 0
nan_cols = []
for index, value in enumerate(train_X.isna().sum()):
    if value != 0:
        s += 1
        print(f.loc[index, 'index'], value)
        nan_cols.append(f.loc[index, 'index'])

print(s)

agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__last__ALL_TIME 9112
agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__mean__ALL_TIME 9112
agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__last__ALL_TIME 10154
agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__mean__ALL_TIME 10154
agg_sroomer__g_contractor__sroomer_id__count__3M 7263
agg_sroomer__g_contractor__sroomer_id__count__6M 7263
agg_sroomer__g_contractor__sroomer_id__count__12M 7263
agg_sroomer__g_contractor__sroomer_id__count__ALL_TIME 7263
agg_FinanceAndTaxesFTS__g_contractor__TaxArrearsSum__last__ALL_TIME 7727
agg_FinanceAndTaxesFTS__g_contractor__TaxPenaltiesSum__last__ALL_TIME 10997
10


In [60]:
f[f['Nans'] != 0]

,index,Nans
10,agg_all_contracts__g_contract__abs_change_pric...,9112
11,agg_all_contracts__g_contract__abs_change_pric...,9112
12,agg_all_contracts__g_contract__rel_change_pric...,10154
13,agg_all_contracts__g_contract__rel_change_pric...,10154
119,agg_sroomer__g_contractor__sroomer_id__count__3M,7263
120,agg_sroomer__g_contractor__sroomer_id__count__6M,7263
121,agg_sroomer__g_contractor__sroomer_id__count__12M,7263
122,agg_sroomer__g_contractor__sroomer_id__count__...,7263
148,agg_FinanceAndTaxesFTS__g_contractor__TaxArrea...,7727
149,agg_FinanceAndTaxesFTS__g_contractor__TaxPenal...,10997


In [61]:
train_X = train_X.drop(columns=nan_cols).reset_index().drop(columns='index')
df = pd.merge(train_X, train_y, how='inner', on=['contract_id', 'report_date'])
dates = np.array(df.dtypes[df.dtypes == 'object'].index.tolist())
df[dates[0]] = pd.to_datetime(df[dates[0]])
df[dates[1]] = pd.to_datetime(df[dates[1]])
X = df[train_X.columns]
y = df[train_y.columns]

In [64]:
y.dtypes

contract_id             int64
report_date    datetime64[ns]
default6                int64
dtype: object

In [74]:
X.dtypes

contract_id                                                                     int64
report_date                                                            datetime64[ns]
specialization_id                                                               int64
contract_init_sum                                                             float64
contract_date                                                          datetime64[ns]
                                                                            ...      
agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W                 float64
agg_tender_proposal__g_contractor__id__ALL__countDistinct__12W                float64
agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W                float64
agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W                float64
agg_tender_proposal__g_contractor__id__ALL__countDistinct__ALL_TIME           float64
Length: 159, dtype: object

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [77]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

parameters = {'max_depth': [4, 5, 6],
              'min_child_weight': [1, 2, 3, 5],
              'learning_rate': [0.03, 0.3, 0.01, 0.5],
              'n_estimators': [100, 250, 500, 1000, 1500],
              'objective': 'binary:logistic'}
XGBoost = xgb.XGBClassifier()
clf = GridSearchCV(XGBoost, parameters, n_jobs=-1, cv=5, scoring='f1')
clf.fit(X, y)

DTypePromotionError: The DType <class 'numpy.dtypes.Int64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int64DType'>)